<a href="https://colab.research.google.com/github/DanielRasho/Paralela-proj3/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tests

## Validación funcional CPU vs GPU (Memoria Global)

En esta sección verificamos que ambas implementaciones de la Transformada de Hough —la secuencial (`cpu.cu`) y la paralela (`globa.cu`)— generan acumuladores equivalentes.

El objetivo es comprobar que la versión CUDA no altera los resultados, garantizando que la paralelización no ha introducido errores lógicos antes de continuar con la medición de rendimiento.


In [ ]:
import subprocess
from pathlib import Path

base_dir = Path("Paralela-proj3-main/src")

cpu_exec = base_dir / "cpu_exec"
gpu_exec = base_dir / "gpu_exec"

# Compilar versión CPU
subprocess.run(["g++", "-O2", str(base_dir / "cpu.cu"), "-o", str(cpu_exec), "-lm", "-lpng"])

# Compilar versión CUDA (memoria global)
subprocess.run(["nvcc", "-O2", str(base_dir / "globa.cu"), "-o", str(gpu_exec), "-lpng"])


In [ ]:
import numpy as np

media_dir = Path("Paralela-proj3-main/media")
input_image = media_dir / "runway.pgm"

cpu_output = media_dir / "cpu_out.png"
gpu_output = media_dir / "gpu_out.png"

# Ejecutar CPU
subprocess.run([str(cpu_exec), str(input_image), str(cpu_output)], check=True)

# Ejecutar GPU (memoria global)
subprocess.run([str(gpu_exec), str(input_image), str(gpu_output)], check=True)

print("Ejecuciones completadas correctamente.")


### Ejecución de ambas versiones

Se ejecutan las dos versiones sobre la misma imagen de entrada (`runway.pgm`).

Ambas producen un archivo PNG con las líneas detectadas.  
En este punto verificamos que el proceso se completa sin errores ni bloqueos, lo cual confirma que las versiones son funcionales y generan acumuladores de tamaño correcto.


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

cpu_img = np.array(Image.open(cpu_output))
gpu_img = np.array(Image.open(gpu_output))


diff = np.abs(cpu_img.astype(int) - gpu_img.astype(int))
mse = np.mean(diff**2)

plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.imshow(cpu_img)
plt.title("CPU Output")

plt.subplot(1,3,2)
plt.imshow(gpu_img)
plt.title("GPU Output")

plt.subplot(1,3,3)
plt.imshow(diff)
plt.title(f"Diferencia (MSE = {mse:.2f})")
plt.show()

print(f"Error cuadrático medio entre CPU y GPU: {mse:.4f}")
